### Preperations and imports

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# change working directory
%cd drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [2]:
import os
import PIL
import cv2
import torch
import random
import pandas as pd
import torch.nn as nn
import torchvision
import torch.nn.functional as f

from tqdm import tqdm
from torch import optim
from pathlib import Path
from datetime import datetime
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize, functional

# hacks for time-saving
torch.backends.cudnn.benchmark = True
torch.autograd.profiler.emit_nvtx(False)
torch.autograd.profiler.profile(False)
torch.autograd.set_detect_anomaly(False)

# if GPU is available, prepare it for heavy calculations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# empty GPU memory
torch.cuda.empty_cache()

In [10]:
# view working directory
os.getcwd()

'/content/drive/MyDrive'

In [3]:
# view GPU info and processes
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



### Deep Learning algorithm

In [4]:
def get_train_test_paths(test_ratio: float = 0.2):
    # extract the data from the dataset folder
    files = [file_name for file_name in
             Path(os.getcwd() + os.sep + 'Water Bodies Dataset' + os.sep + 'Images').rglob("*.jpg")]
    # randomize the order of the data
    random.shuffle(files)
    # separate test and train files
    first_train = int(test_ratio * len(files))
    test_path = files[:first_train]
    train_path = files[first_train:]
    return train_path, test_path


def get_mask_path(file_path):
    # gets source image path, returns mask path
    file_path = str(file_path).replace('Images', 'Masks')
    return file_path


class WaterDataset(Dataset):
    def __init__(self, path_list, transform_source=None, transform_both=None):
        self.sources = path_list
        self.transform_source = transform_source
        self.transform_both = transform_both

    def __len__(self):
        return len(self.sources)

    def __getitem__(self, index):
        img_path = self.sources[index]
        img_num = int(str(img_path).rsplit('_', 1)[1].split('.')[0])
        source = functional.to_tensor(PIL.Image.open(img_path))
        label = functional.to_tensor(PIL.Image.open(get_mask_path(img_path)).convert('L'))

        if self.transform_source:
            source = self.transform_source(source)
        if self.transform_both:
            source = self.transform_both(source)
            label = self.transform_both(label)
            label = (label < 0.5).float()

            assert len(label.unique()) <= 2, "threshold didn't work"

        return source, label, img_num


def get_train_test_loaders(batch_size, length):
    train_path, test_path = get_train_test_paths()
    train_loader = DataLoader(dataset=WaterDataset(train_path,
                                                   transform_both=torchvision.transforms.Resize((length, length))),
                              batch_size=batch_size,
                              pin_memory=True,
                              num_workers=1,
                              shuffle=True)
    test_loader = DataLoader(dataset=WaterDataset(tuple(test_path),
                                                  transform_both=torchvision.transforms.Resize((length, length))),
                             batch_size=batch_size,
                             pin_memory=True,
                             num_workers=1,
                             shuffle=False)

    return train_loader, test_loader, test_path

In [5]:
class Hidden1(nn.Module):
    def __init__(self, length, hidden_size, activation):
        super().__init__()
        self.activation = activation
        self.length = length

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(length**2 * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, length**2 * 2)  # 2 for each class

    def forward(self, x):
        x = self.flat(x)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x


class Hidden2(nn.Module):
    def __init__(self, length, hidden_size, activation):
        super().__init__()
        self.activation = activation
        self.length = length

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(length**2 * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, length**2 * 2)  # 2 for each class

    def forward(self, x):
        x = self.flat(x)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x


class Conv1(nn.Module):
    def __init__(self, length, hidden_size, activation, kernel_size: int = 3):
        super().__init__()
        self.activation = activation
        self.length = length
        self.kernel_size = kernel_size  # expected odd kernel_size
        self.hidden_size = hidden_size

        self.conv1 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.fc1 = nn.Linear(length**2 * 3, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, length**2 * 2)

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x


class Conv2(nn.Module):
    def __init__(self, length, hidden_size, activation, kernel_size: int = 3):
        super().__init__()
        self.activation = activation
        self.length = length
        self.kernel_size = kernel_size  # expected odd kernel_size
        self.hidden_size = hidden_size

        self.conv1 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.conv2 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.fc1 = nn.Linear(length**2 * 3, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, length**2 * 2)

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = self.activation(self.conv2(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x


class Conv3(nn.Module):
    def __init__(self, length, hidden_size, activation, kernel_size: int = 3):
        super().__init__()
        self.activation = activation
        self.length = length
        self.kernel_size = kernel_size  # expected odd kernel_size
        self.hidden_size = hidden_size

        self.conv1 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.conv2 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.conv3 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.fc1 = nn.Linear(length**2 * 3, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, length**2 * 2)

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = self.activation(self.conv2(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = self.activation(self.conv3(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x

In [6]:
def fit_model(model, model_parameters, loss_function, optimizer, batch_size, image_normalized_length, num_of_epochs):
    train_loader, test_loader, _ = get_train_test_loaders(batch_size, image_normalized_length)
    # if GPU is available, prepare it for heavy calculations
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model(*model_parameters).to(device)
    criterion = loss_function()
    optimizer = optimizer(model.parameters(), lr=3e-4)
    for epoch in range(1, num_of_epochs + 1):
        model.train()
        train_start = datetime.now()
        epoch_loss = 0
        for image, mask, _ in tqdm(train_loader):
            x = image.float().to(device)
            tag = mask.flatten().long().to(device)
            optimizer.zero_grad()
            prediction = model(x)
            loss = criterion(prediction, tag)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        train_end = datetime.now()
        train_seconds = (train_end - train_start).total_seconds()
        model.eval()
        test_start = datetime.now()
        total_predicted_positive, total_true_positive, total_false_negative, \
        total_true_prediction, total_false_prediction = 0, 0, 0, 0, 0
        with torch.no_grad():
            for x, y, img_nums in tqdm(test_loader):
                img_nums = img_nums.tolist()
                x = x.float().to(device)
                y = y.flatten().long().to(device)
                probabilities = model(x)
                prediction = torch.argmax(probabilities, dim=1).float()

                for i in range(0, len(img_nums)):
                    img_num = img_nums[i]
                    if img_num in [19, 22, 699, 9]:
                        pred = torch.split(prediction, [image_normalized_length**2 for _ in range(batch_size)])[i]
                        out_path = f'out/water_body_{img_num}.jpg'
                        prediction = prediction.reshape(100, 100)
                        save_image(prediction, out_path)
                
                predicted_positive = (prediction == 1).sum().item()
                true_positive = ((prediction == 1) & (y == 1)).sum().item()
                false_negative = ((prediction == 0) & (y == 1)).sum().item()

                total_predicted_positive += predicted_positive
                total_true_positive += true_positive
                total_false_negative += false_negative
                total_true_prediction += (prediction == y).sum().item()
                total_false_prediction += (prediction != y).sum().item()
        test_end = datetime.now()
        test_seconds = (test_end - test_start).total_seconds()
        recall = total_true_positive / (total_true_positive + total_false_negative)
        precision = total_true_positive / total_predicted_positive
        f1 = (2 * precision * recall) / (precision + recall)
        accuracy = total_true_prediction / (total_true_prediction + total_false_prediction)
        # append results to csv file
        df = pd.DataFrame({'Model Name': [model.__class__.__name__],
                           'Iteration': [epoch],
                           'Input Image Length': [image_normalized_length],
                           'Hidden Layer Size': [hidden_layer_size],
                           'Batch Size': [batch_size],
                           'Activation Function': [str(model_parameters[2].__name__)],
                           'Optimizer': [str(type(optimizer))],
                           'Loss Function': [str(loss_function)],
                           'Loss': [epoch_loss],
                           'Recall': [recall],
                           'Precision': [precision],
                           'F1': [f1],
                           'Accuracy': [accuracy],
                           'Iteration Training Seconds': [train_seconds],
                           'Iteration Testing Seconds': [test_seconds]})
        df.to_csv('Water_Bodies_Results.csv', index=False, mode='a', header=False)
        print(df)
    # save the trained model
    model_path = f'{os.getcwd()}{os.sep}{model.__class__.__name__}-{str(model_parameters[2].__name__)}-{hidden_layer_size}.pt'
    torch.save(model.state_dict(), model_path)
    # clean memory of current model on exit
    torch.cuda.empty_cache()


if __name__ == '__main__':
    models = (Hidden1, Hidden2, Conv1, Conv2, Conv3)
    activation_funcs = (f.relu, f.leaky_relu)
    hidden_layer_sizes = (4000, 3000, 2000)

    optimizer = optim.Adam
    loss_func = nn.CrossEntropyLoss
    image_normalized_length = 100
    batch_size = 16
    num_of_epochs = 10
    kernel_size = 3

    # train models with varying hyperparameters
    for model in models:
        for activation_func in activation_funcs:
            for hidden_layer_size in hidden_layer_sizes:
                if 'Conv' in model.__name__:
                    model_parameters = (image_normalized_length, hidden_layer_size, activation_func, kernel_size)
                model_parameters = (image_normalized_length, hidden_layer_size, activation_func)
                fit_model(model, model_parameters, loss_func, optimizer,
                          batch_size, image_normalized_length, num_of_epochs)

  3%|▎         | 1/36 [00:13<08:00, 13.73s/it]


RuntimeError: ignored

### Save predictions and print # of parameters

In [ ]:
# # save model parameters
# torch.save(the_model.state_dict(), PATH)
# # load model parameters
# the_model = TheModelClass(*args, **kwargs)
# the_model.load_state_dict(torch.load(PATH))

# # save the entire model
# torch.save(the_model, PATH)
# # load the entire model
# the_model = torch.load(PATH)

In [ ]:
model = Conv1(100, 3000, f.leaky_relu).to(device)
model_path = f'{os.getcwd()}{os.sep}Conv1-leaky_relu-3.pt'
model.load_state_dict(torch.load(model_path))
_, test_loader, path_list = get_train_test_loaders(1, 100)

with torch.no_grad():
    for image, mask, img_num in test_loader:
        img_num = str(img_num.item())
        out_path = f'out/water_body_{img_num}.jpg'
        x = image.float().to(device)
        tag = mask.flatten().long().to(device)
        prediction = model(x).float()
        prediction = torch.argmax(prediction, dim=1)
        prediction = prediction.reshape(100, 100)
        # get original size
        # img = cv2.imread(out_path)
        # height, width = img.shape[0], img.shape[1]
        # resize prediction to original source size
        # prediction = f.interpolate(prediction, size=(height, width))
        # save the prediction
        save_image(prediction, out_path)

In [ ]:
def get_n_params(model):
    """https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/6"""
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn * s
        pp += nn
    return pp

model = Hidden1(100, 4000, f.relu).to(device)
model_path = f'{os.getcwd()}{os.sep}Hidden1-relu-4.pt'
model.load_state_dict(torch.load(model_path))

with torch.no_grad():
  print(get_n_params(model))

### **WARNING:** Wipe results file

In [24]:
# # uncomment all lines below to run
# df = pd.DataFrame({'Model Name': [],
#                    'Iteration': [],
#                    'Input Image Length': [],
#                    'Hidden Layer Size': [],
#                    'Batch Size': [],
#                    'Activation Function': [],
#                    'Optimizer': [],
#                    'Loss Function': [],
#                    'Loss': [],
#                    'Recall': [],
#                    'Precision': [],
#                    'F1': [],
#                    'Accuracy': [],
#                    'Iteration Training Seconds': [],
#                    'Iteration Testing Seconds': []})
# df.to_csv('Water_Bodies_Results.csv', index=False, header=True)